# 1 Classification



# Step 1: Preparing Data 

In [33]:
import sys 
import numpy as np
import pandas as pd
import statsmodels.api as sm
import sklearn
import scipy as sp
%matplotlib inline 
import matplotlib.pyplot as plt
from sklearn.metrics import fbeta_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import xgboost as xgb
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV


In [35]:
df=pd.read_csv('dataframe_train.csv')
df_p=pd.read_csv('dataframe_test.csv')

In [37]:
# for df
columns_all=df.columns.values
columns_need=['grid_distance','target_lng','target_lat','action_type','weather_grade','courier_wave_start_lng','courier_wave_start_lat','group','level','speed','max_load','urgency','hour','source_lng','source_lat']
columns_to_drop=[]
for i in columns_all:
    if i not in columns_need:
        columns_to_drop.append(i)
print(columns_to_drop)
df=df.drop(columns=columns_to_drop)

['courier_id', 'wave_index', 'tracking_id', 'date', 'aoi_id', 'shop_id', 'id', 'source_type', 'source_tracking_id', 'expected_use_time']


In [39]:
#for df_p
columns_all=df_p.columns.values

columns_need=['grid_distance','target_lng','target_lat','action_type','weather_grade','courier_wave_start_lng','courier_wave_start_lat','group','level','speed','max_load','urgency','hour','source_lng','source_lat']
columns_to_drop=[]
for i in columns_all:
    if i not in columns_need:
        columns_to_drop.append(i)
print(columns_to_drop)
df_p=df_p.drop(columns=columns_to_drop)

['courier_id', 'wave_index', 'tracking_id', 'date', 'aoi_id', 'shop_id', 'id', 'source_type', 'source_tracking_id']


In [40]:
print(df.info())
print(df.head())
print(df_p.info())
print(df_p.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509604 entries, 0 to 509603
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   courier_wave_start_lng  509604 non-null  float64
 1   courier_wave_start_lat  509604 non-null  float64
 2   action_type             509604 non-null  object 
 3   group                   509604 non-null  float64
 4   level                   509604 non-null  int64  
 5   speed                   509604 non-null  float64
 6   max_load                509604 non-null  int64  
 7   weather_grade           509604 non-null  object 
 8   source_lng              509604 non-null  float64
 9   source_lat              509604 non-null  float64
 10  target_lng              509604 non-null  float64
 11  target_lat              509604 non-null  float64
 12  grid_distance           509604 non-null  float64
 13  urgency                 509604 non-null  int64  
 14  hour                

In [41]:
df=pd.get_dummies(df,columns=['action_type','weather_grade'],drop_first=True)
# df=pd.get_dummies(df,columns=['source_type'],drop_first=True)
df_p=pd.get_dummies(df_p,columns=['weather_grade'],drop_first=True)
print(df.info())
print(df_p.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509604 entries, 0 to 509603
Data columns (total 17 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   courier_wave_start_lng              509604 non-null  float64
 1   courier_wave_start_lat              509604 non-null  float64
 2   group                               509604 non-null  float64
 3   level                               509604 non-null  int64  
 4   speed                               509604 non-null  float64
 5   max_load                            509604 non-null  int64  
 6   source_lng                          509604 non-null  float64
 7   source_lat                          509604 non-null  float64
 8   target_lng                          509604 non-null  float64
 9   target_lat                          509604 non-null  float64
 10  grid_distance                       509604 non-null  float64
 11  urgency                   

In [42]:
from sklearn.model_selection import train_test_split as tr_te_split
X = df.drop(columns=['action_type_PICKUP'])
y = df['action_type_PICKUP'].ravel()
X_train, X_test, y_train, y_test = tr_te_split(X, y, test_size=0.3,random_state=0)

In [43]:
X.head()

,courier_wave_start_lng,courier_wave_start_lat,group,level,speed,max_load,source_lng,source_lat,target_lng,target_lat,grid_distance,urgency,hour,weather_grade_Normal Weather,weather_grade_Slightly Bad Weather,weather_grade_Very Bad Weather
0,121.630997,39.142343,2.020020e+16,3,4.751832,11,121.630997,39.142343,121.632547,39.141946,377.0,1246,11,1,0,0
1,121.630997,39.142343,2.020020e+16,3,4.751832,11,121.632547,39.141946,121.626144,39.140281,780.0,1246,11,1,0,0
2,121.630997,39.142343,2.020020e+16,3,4.751832,11,121.626144,39.140281,121.631219,39.141811,550.0,2462,11,1,0,0
3,121.630997,39.142343,2.020020e+16,3,4.751832,11,121.631219,39.141811,121.632084,39.146201,707.0,1205,11,1,0,0
4,121.630997,39.142343,2.020020e+16,3,4.751832,11,121.632084,39.146201,121.631574,39.142231,770.0,1882,11,1,0,0


In [44]:
df_p.head()

,courier_wave_start_lng,courier_wave_start_lat,group,level,speed,max_load,source_lng,source_lat,target_lng,target_lat,grid_distance,urgency,hour,weather_grade_Normal Weather,weather_grade_Slightly Bad Weather,weather_grade_Very Bad Weather
0,121.630558,39.1409,2.020000e+16,3,5.535851,11,121.630558,39.140900,121.629930,39.144603,796.0,1776,13,1,0,0
1,121.630558,39.1409,2.020000e+16,3,5.535851,11,121.629930,39.144603,121.629604,39.150661,1048.0,697,13,1,0,0
2,121.630558,39.1409,2.020000e+16,3,5.535851,11,121.629604,39.150661,121.629930,39.144603,1019.0,1756,13,1,0,0
3,121.630558,39.1409,2.020000e+16,3,5.535851,11,121.629930,39.144603,121.630544,39.152711,1198.0,676,13,1,0,0
4,121.630558,39.1409,2.020000e+16,3,5.535851,11,121.630544,39.152711,121.631446,39.142762,1275.0,2336,14,1,0,0


In [45]:
#Scale Necessary Set
scaler = StandardScaler().fit(X_train)
X_train_st = scaler.transform(X_train)
X_test_st = scaler.transform(X_test)

scaler_p = StandardScaler().fit(X)
df_p = scaler_p.transform(df_p)
X_st=scaler_p.transform(X)

# Step 2: Train on different baseline models

In [46]:
# # Fit a Baseline Model with all features
# # Logistic Classification
# from sklearn.linear_model import LogisticRegression as LogitReg
# LR=LogitReg(max_iter=10000).fit(X_train_st,y_train)
# prob_pred=LR.predict_proba(X_train_st)
# print(prob_pred)

# # t=0.5
# # y_pred=np.array(list(map(lambda x: int(x>=t), prob_pred[:,1])))
# # y_pred

# # from sklearn.metrics import auc,roc_curve
# # Out-of-Sample
# # fpr,tpr,thresholds=roc_curve(y_test,prob_test[:,1])
# # roc_auc=auc(fpr,tpr)
# # print("The out of sample AUC is {:4f}.".format(roc_auc))
# prob_test=LR.predict_proba(X_test_st)
# y_pred_test=list(map(lambda x: int(x>=0.7),prob_test[:,1]))
# fbeta=fbeta_score(y_test, y_pred_test, beta=0.5)
# print('The Fbeta_score for Logistic Classification Model is', fbeta)
# # The importance of each feature

In my previous trials: 
The Fbeta_score for Logistic Classification Model is 0.6880545634746095

In [47]:
# # Fit a Baseline Model with all features
# # Decision Tree
# from sklearn.tree import DecisionTreeClassifier,export_graphviz
# from sklearn.metrics import accuracy_score
# Action_type_tree=DecisionTreeClassifier()
# Action_type_tree.fit(X_train_st,y_train)
# y_pred=Action_type_tree.predict(X_test_st)
# # y_score=Action_type_tree.predict_proba(X_test_st)
# # fpr,tpr,threshold= roc_curve(y_test,y_score[:,1])
# # roc_auc=auc(fpr,tpr)
# fbeta=fbeta_score(y_test, y_pred, beta=0.5)
# print('The Fbeta_score for Decision Tree is', fbeta)
# # The importance of each feature

In my previous trials: The Fbeta_score for Decision Tree is 0.7186692799937696

In [48]:
# # Fit a Baseline Model with all features
# # KNN Classification
# from sklearn.neighbors import KNeighborsClassifier
# Action_Type_knn=KNeighborsClassifier(n_neighbors=100).fit(X_train_st,y_train)
# y_pred = Action_Type_knn.predict(X_test_st)
# # pred_prob=Action_Type_knn.predict_proba(X_test_st)
# # fpr,tpr,threshold=roc_curve(y_test,pred_prob[:,1])
# # roc_auc=auc(fpr,tpr)
# # print("The out-of-sample AUC for KNN is", roc_auc)
# fbeta=fbeta_score(y_test, y_pred, beta=0.5)
# print('The Fbeta_score for KNN Classification is', fbeta)
# # The importance of each feature


In previous trials:The Fbeta_score for KNN Classification is 0.8083948817076496

In [49]:
# # Fit a Baseline Model with all features
# # XgBoost
# import xgboost as xgb
# data_train=xgb.DMatrix(data=X_train_st,label=y_train)
# data_test=xgb.DMatrix(data=X_test_st,label=y_test)

# Action_type_xgbt=xgb.XGBClassifier().fit(X_train_st,y_train)

# y_pred=Action_type_xgbt.predict(X_test_st)
# fbeta=fbeta_score(y_test, y_pred, beta=0.5)
# print('The Fbeta_score for XgBoost is', fbeta)
# print(X_train.columns.values)
# print('The features importance is shown here:',Action_type_xgbt.feature_importances_ )

In previous trials: The Fbeta_score for XgBoost is 0.8124462981252886

From the Fbeta_score listed above, xgBoost should be the best model. Further fine tune for it.

# Step 2(b): Narrow down the feature combination using the best model selected

Select the features that have importance > 0.03

In [50]:
df=pd.read_csv('dataframe_train.csv')
df_p=pd.read_csv('dataframe_test.csv')

In [51]:
# for df
columns_all=df.columns.values
columns_need=['courier_wave_start_lng','courier_wave_start_lat','target_lng','target_lat','grid_distance','urgency','action_type']

columns_to_drop=[]
for i in columns_all:
    if i not in columns_need:
        columns_to_drop.append(i)
print(columns_to_drop)
df=df.drop(columns=columns_to_drop)

['courier_id', 'wave_index', 'tracking_id', 'date', 'group', 'level', 'speed', 'max_load', 'weather_grade', 'aoi_id', 'shop_id', 'id', 'source_type', 'source_tracking_id', 'source_lng', 'source_lat', 'expected_use_time', 'hour']


In [52]:
#for df_p
columns_all=df_p.columns.values

columns_need=['courier_wave_start_lng','courier_wave_start_lat','target_lng','target_lat','grid_distance','urgency','action_type']
columns_to_drop=[]
for i in columns_all:
    if i not in columns_need:
        columns_to_drop.append(i)
print(columns_to_drop)
df_p=df_p.drop(columns=columns_to_drop)

['courier_id', 'wave_index', 'tracking_id', 'date', 'group', 'level', 'speed', 'max_load', 'weather_grade', 'aoi_id', 'shop_id', 'id', 'source_type', 'source_tracking_id', 'source_lng', 'source_lat', 'hour']


In [53]:
df=pd.get_dummies(df,columns=['action_type'],drop_first=True)
print(df.info())
print(df_p.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509604 entries, 0 to 509603
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   courier_wave_start_lng  509604 non-null  float64
 1   courier_wave_start_lat  509604 non-null  float64
 2   target_lng              509604 non-null  float64
 3   target_lat              509604 non-null  float64
 4   grid_distance           509604 non-null  float64
 5   urgency                 509604 non-null  int64  
 6   action_type_PICKUP      509604 non-null  uint8  
dtypes: float64(5), int64(1), uint8(1)
memory usage: 23.8 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25468 entries, 0 to 25467
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   courier_wave_start_lng  25468 non-null  float64
 1   courier_wave_start_lat  25468 non-null  float64
 2   target_lng   

In [54]:
from sklearn.model_selection import train_test_split as tr_te_split
X = df.drop(columns=['action_type_PICKUP'])
y = df['action_type_PICKUP'].ravel()
X_train, X_test, y_train, y_test = tr_te_split(X, y, test_size=0.3,random_state=0)

#Scale Necessary Set
scaler = StandardScaler().fit(X_train)
X_train_st = scaler.transform(X_train)
X_test_st = scaler.transform(X_test)

scaler_p = StandardScaler().fit(X)
df_p = scaler_p.transform(df_p)
X_st=scaler_p.transform(X)

In [55]:
# Fit a Baseline Model with all features
# XgBoost
import xgboost as xgb
data_train=xgb.DMatrix(data=X_train_st,label=y_train)
data_test=xgb.DMatrix(data=X_test_st,label=y_test)

Action_type_xgbt=xgb.XGBClassifier().fit(X_train_st,y_train)

y_pred=Action_type_xgbt.predict(X_test_st)
fbeta=fbeta_score(y_test, y_pred, beta=0.5)
print('The Fbeta_score for XgBoost is', fbeta)
print(X_train.columns.values)
print('The features importance is shown here:',Action_type_xgbt.feature_importances_ )

C:\Users\gzLij\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[18:46:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
The Fbeta_score for XgBoost is 0.8428487284676511
['courier_wave_start_lng' 'courier_wave_start_lat' 'target_lng'
 'target_lat' 'grid_distance' 'urgency']
The features importance is shown here: [0.08255818 0.

# Step 3: Fine Tune after the best model and best feature combinations found

I tried to use PCA and CV, but none of them give a good result comparing to the simplest method

In [56]:
import xgboost as xgb
data_train=xgb.DMatrix(data=X_train_st,label=y_train)
data_test=xgb.DMatrix(data=X_test_st,label=y_test)

clf=xgb.XGBClassifier(gamma=2,max_depth=12,learning_rate=1,n_estimators=120)
poly_pipe=make_pipeline(clf)
Action_type_xgbt=poly_pipe.fit(X_train_st,y_train)
y_pred=Action_type_xgbt.predict(X_test_st)
fbeta=fbeta_score(y_test, y_pred, beta=0.5)
print('The Fbeta_score for XgBoost is', fbeta)
print(X_train.columns.values)
print('The features importance is shown here:',clf.feature_importances_ )

C:\Users\gzLij\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[18:46:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
The Fbeta_score for XgBoost is 0.9396499026068216
['courier_wave_start_lng' 'courier_wave_start_lat' 'target_lng'
 'target_lat' 'grid_distance' 'urgency']
The features importance is shown here: [0.08957425 0.

Feature Importance Analysis:
        
        It seems that the source type highly indicates the next action of the courier. Mathmatically, it has at least 1/3 probability of getting the right answer. So it has the highest importance.  

# Step 4: Retrain & Step 5: Predict

In [58]:
data_train=xgb.DMatrix(data=X_st,label=y)

Finalized_poly_pipe=make_pipeline(xgb.XGBClassifier(gamma=2,max_depth=12,learning_rate=1,n_estimators=120))
Finalized_poly_pipe.fit(X_st,y)
prediction=1-Finalized_poly_pipe.predict(df_p)
prediction

[18:48:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


array([0, 1, 0, ..., 1, 0, 1], dtype=uint8)

In [62]:
# I use 1-Finalized_poly_pipe.predict(df_p) because I got 'action_type_pickup' when I dummy variables. Therefore, I will need to make a change before exporting data to the file.

In [63]:
prediction=pd.DataFrame(prediction)
prediction.describe()

,0
count,25468.000000
mean,0.498822
std,0.500008
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [64]:
df_write= pd.read_csv('Classification.csv')
df_write['action_type_DELIVERY'] = prediction
df_write.to_csv(r'Classification_sub.csv', index = False, header=True)







# 2 Regression 

# Step 1: Preparing Data

In [73]:
import sys 
import numpy as np
import pandas as pd
import statsmodels.api as sm
import sklearn
import scipy as sp
%matplotlib inline 
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error as MAE 

In [74]:
df=pd.read_csv('dataframe_train.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509604 entries, 0 to 509603
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   courier_id              509604 non-null  int64  
 1   wave_index              509604 non-null  int64  
 2   tracking_id             509604 non-null  float64
 3   courier_wave_start_lng  509604 non-null  float64
 4   courier_wave_start_lat  509604 non-null  float64
 5   action_type             509604 non-null  object 
 6   date                    509604 non-null  int64  
 7   group                   509604 non-null  float64
 8   level                   509604 non-null  int64  
 9   speed                   509604 non-null  float64
 10  max_load                509604 non-null  int64  
 11  weather_grade           509604 non-null  object 
 12  aoi_id                  509604 non-null  object 
 13  shop_id                 509604 non-null  object 
 14  id                  

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,date,group,level,speed,...,source_type,source_tracking_id,source_lng,source_lat,target_lng,target_lat,grid_distance,expected_use_time,urgency,hour
0,10007871,0,2.100070e+18,121.630997,39.142343,PICKUP,20200201,2.020020e+16,3,4.751832,...,ASSIGN,2.100070e+18,121.630997,39.142343,121.632547,39.141946,377.0,804,1246,11
1,10007871,0,2.100070e+18,121.630997,39.142343,DELIVERY,20200201,2.020020e+16,3,4.751832,...,PICKUP,2.100070e+18,121.632547,39.141946,121.626144,39.140281,780.0,298,1246,11
2,10007871,0,2.100070e+18,121.630997,39.142343,PICKUP,20200201,2.020020e+16,3,4.751832,...,DELIVERY,2.100070e+18,121.626144,39.140281,121.631219,39.141811,550.0,545,2462,11
3,10007871,0,2.100070e+18,121.630997,39.142343,DELIVERY,20200201,2.020020e+16,3,4.751832,...,PICKUP,2.100070e+18,121.631219,39.141811,121.632084,39.146201,707.0,341,1205,11
4,10007871,0,2.100070e+18,121.630997,39.142343,PICKUP,20200201,2.020020e+16,3,4.751832,...,DELIVERY,2.100070e+18,121.632084,39.146201,121.631574,39.142231,770.0,166,1882,11


In [75]:
df=pd.get_dummies(df,columns=['source_type','action_type','weather_grade'],drop_first=True)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509604 entries, 0 to 509603
Data columns (total 28 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   courier_id                          509604 non-null  int64  
 1   wave_index                          509604 non-null  int64  
 2   tracking_id                         509604 non-null  float64
 3   courier_wave_start_lng              509604 non-null  float64
 4   courier_wave_start_lat              509604 non-null  float64
 5   date                                509604 non-null  int64  
 6   group                               509604 non-null  float64
 7   level                               509604 non-null  int64  
 8   speed                               509604 non-null  float64
 9   max_load                            509604 non-null  int64  
 10  aoi_id                              509604 non-null  object 
 11  shop_id                   

In [76]:
columns_all=df.columns.values
print(columns_all)

columns_need=['wave_index','courier_wave_start_lng','courier_wave_start_lat','group','level','grid_distance','urgency','hour','expected_use_time','speed','weather_grade_Slightly Bad Weather','weather_grade_Very Bad Weather','source_type_DELIVERY','source_type_PICKUP','weather_grade_Normal Weather']

columns_to_drop=[]
for i in columns_all:
    if i not in columns_need:
        columns_to_drop.append(i)

df=df.drop(columns=columns_to_drop)

['courier_id' 'wave_index' 'tracking_id' 'courier_wave_start_lng'
 'courier_wave_start_lat' 'date' 'group' 'level' 'speed' 'max_load'
 'aoi_id' 'shop_id' 'id' 'source_tracking_id' 'source_lng' 'source_lat'
 'target_lng' 'target_lat' 'grid_distance' 'expected_use_time' 'urgency'
 'hour' 'source_type_DELIVERY' 'source_type_PICKUP' 'action_type_PICKUP'
 'weather_grade_Normal Weather' 'weather_grade_Slightly Bad Weather'
 'weather_grade_Very Bad Weather']


Data Choice Analysis:

    grid-distance/speed: In the simplest case, Time=Distance/Speed
    urgency: Higher urgency may indicate shorter time (the courier may speed up)
    hour:Courier are busy in different period of the day. The less busy they are, I assume their deliveries to be faster.
    weather_grade: bad weather slow down the speed and thus take longer time to deliver.


In [77]:
# Prove to be a bad strategy:
# df['busy_hour']=np.array((df['hour']>=10) & (df['hour']<=19)).astype(int)
# df=df.drop(columns=['hour'])
# df[df['busy_hour']==1].info()

In [78]:
from sklearn.model_selection import train_test_split as tr_te_split
X = df.drop(columns=['expected_use_time'])
y = df['expected_use_time'].ravel()
X_train, X_test, y_train, y_test = tr_te_split(X, y, test_size=0.3,random_state=0)

In [79]:
# Standardize both the training and the testing sets.
# Using StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train_st = scaler.transform(X_train)
X_test_st = scaler.transform(X_test)

In [80]:
# from sklearn.preprocessing import MinMaxScaler
# scaler=MinMaxScaler()
# scaler.fit(X_train)
# X_train_st = scaler.transform(X_train)
# X_test_st = scaler.transform(X_test)

# Train On 4 different Models and See their performances 
(LR/KNN/TREE/RANDOM FOREST/XGBOOST)

In [81]:
# # Fit a Baseline Model with all features
# # Logistic Regression
# from sklearn.linear_model import LogisticRegression as LogitReg
# LR=LogitReg(max_iter=10000).fit(X_train_st,y_train)
# prob_pred=LR.predict_proba(X_train_st)
# print(prob_pred)

# prob_test=LR.predict_proba(X_test_st)
# y_pred_test=list(map(lambda x: int(x>=0.5),prob_test[:,1]))
# MAE_score=MAE(y_test, y_pred_test)
# print('The MAE score for Logistic Regression Model is', MAE)
# # The importance of each feature

In [82]:
# # Fit a Baseline Model with all features
# # KNN Regressor
# from sklearn.neighbors import KNeighborsRegressor
# EUT_knn=KNeighborsRegressor(n_neighbors=100).fit(X_train_st,y_train)
# y_pred=EUT_knn.predict(X_test_st)
# MAE_score=MAE(y_test, y_pred)
# MAE_score

In [83]:
# # Fit a Baseline Model with all features
# # Regression Tree Model
# from sklearn.tree import DecisionTreeRegressor
# EUT_tree = DecisionTreeRegressor().fit(X_train_st,y_train)
# y_pred=EUT_tree.predict(X_test_st)
# MAE_score=MAE(y_test, y_pred)
# MAE_score
# # print('The MAE score for Tree Regression Model is', MAE)

In [84]:
# # Fit a Baseline Model with all features
# # XG Boost
# import xgboost as xgb
# data_train=xgb.DMatrix(data=X_train_st,label=y_train)
# data_test=xgb.DMatrix(data=X_test_st,label=y_test)

# xgb_reg=xgb.XGBRegressor()

# xgb_reg.fit(X_train_st,y_train)
# y_pred=xgb_reg.predict(X_test_st)
# MAE_score=MAE(y_test, y_pred)
# MAE_score
# # print('The MAE score for XgBoost Regression Model is', MAE)

In [85]:
import xgboost as xgb
data_train=xgb.DMatrix(data=X_train_st,label=y_train)
data_test=xgb.DMatrix(data=X_test_st,label=y_test)


xgb_reg=xgb.XGBRegressor(objective='reg:squarederror',colsample_bynode=0.8, gamma=0.001,max_depth=5,n_estimators=20)

xgb_reg.fit(X_train_st,y_train)
y_pred=xgb_reg.predict(X_test_st)
MAE_score=MAE(y_test, y_pred)
print(MAE_score)
print(X_train.columns.values)
print('feature importance: ' ,xgb_reg.feature_importances_)

201.55847429384917
['wave_index' 'courier_wave_start_lng' 'courier_wave_start_lat' 'group'
 'level' 'speed' 'grid_distance' 'urgency' 'hour' 'source_type_DELIVERY'
 'source_type_PICKUP' 'weather_grade_Normal Weather'
 'weather_grade_Slightly Bad Weather' 'weather_grade_Very Bad Weather']
feature importance:  [0.01206657 0.0053394  0.00683972 0.00314411 0.00309758 0.01142361
 0.20503186 0.09495062 0.01257664 0.47133318 0.15356338 0.00450212
 0.00287434 0.01325687]


Model Analysis:

        MAE Score Rank: Logistic Regression > Tree Regressor > Random Forest ≈ KNN > Xgboost
        Directly choose from KNN and Xgboost. Xgboost performs the best, meaning that it simulates the original data generating process the best. Then I fine tune of the model on that.

# Narrow Features based on their significance in the best baseline model

In [86]:
# df=pd.read_csv('dataframe_train.csv')
# df.info()
# df.head()

In [87]:
# df=pd.get_dummies(df,columns=['source_type','action_type','weather_grade'],drop_first=True)
# df.head()
# df.info()

In [88]:
# columns_all=df.columns.values
# print(columns_all)

# columns_need=['wave_index','speed','grid_distance','urgency','hour','source_type_DELIVERY','source_type_PICK_UP','weather_grade_Very Bad Weather','expected_use_time']

# columns_to_drop=[]
# for i in columns_all:
#     if i not in columns_need:
#         columns_to_drop.append(i)

# df=df.drop(columns=columns_to_drop)

In [89]:
# from sklearn.model_selection import train_test_split as tr_te_split
# X = df.drop(columns=['expected_use_time'])
# y = df['expected_use_time'].ravel()
# X_train, X_test, y_train, y_test = tr_te_split(X, y, test_size=0.3,random_state=0)

In [90]:
# # Standardize both the training and the testing sets.
# # Using StandardScaler
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler().fit(X_train)
# X_train_st = scaler.transform(X_train)
# X_test_st = scaler.transform(X_test)

In [91]:
# import xgboost as xgb
# data_train=xgb.DMatrix(data=X_train_st,label=y_train)
# data_test=xgb.DMatrix(data=X_test_st,label=y_test)

In [92]:
# xgb_reg=xgb.XGBRegressor(objective='reg:squarederror',colsample_bynode=0.8, gamma=0.001,max_depth=5,n_estimators=20)

In [93]:
# xgb_reg.fit(X_train_st,y_train)
# y_pred=xgb_reg.predict(X_test_st)
# MAE_score=MAE(y_test, y_pred)
# print(MAE_score)
# print(X_train.columns.values)
# print('feature importance: ' ,xgb_reg.feature_importances_)

This is not as good as the original model, abandon this section

# Fine tune for the Best Model

1. I tried PCA and polynomial features on the baseline model, and find that they did not overperform the baseline model, so 


2. I decided that I should "back to normal". And since using grid_search was too slow (I've tried once), so I finally decided to test on different parameters manually.

In [94]:
# # Fit a Baseline Model with all features
# # XG Boost
# import xgboost as xgb
# data_train=xgb.DMatrix(data=X_train_st,label=y_train)
# data_test=xgb.DMatrix(data=X_test_st,label=y_test)


# xgb_reg=xgb.XGBRegressor(objective='reg:squarederror',colsample_bynode=0.8, gamma=0.001,max_depth=5,n_estimators=20)

# xgb_reg.fit(X_train_st,y_train)
# y_pred=xgb_reg.predict(X_test_st)
# MAE_score=MAE(y_test, y_pred)
# MAE_score
# # print('The MAE score for XgBoost Regression Model is', MAE)

In [95]:
# # Fit a Baseline Model with all features
# # XG Boost
# import xgboost as xgb
# data_train=xgb.DMatrix(data=X_train_st,label=y_train)
# data_test=xgb.DMatrix(data=X_test_st,label=y_test)

# xgb_reg=xgb.XGBRegressor(objective='reg:squarederror',colsample_bynode=0.8, gamma=0.001,max_depth=6,n_estimators=20)

# xgb_reg.fit(X_train_st,y_train)
# y_pred=xgb_reg.predict(X_test_st)
# MAE_score=MAE(y_test, y_pred)
# MAE_score
# # print('The MAE score for XgBoost Regression Model is', MAE)

In [96]:
# # Fit a Baseline Model with all features
# # XG Boost
# import xgboost as xgb
# data_train=xgb.DMatrix(data=X_train_st,label=y_train)
# data_test=xgb.DMatrix(data=X_test_st,label=y_test)

# xgb_reg=xgb.XGBRegressor(colsample_bynode=0.8, gamma=0.001,max_depth=6,n_estimators=5)

# xgb_reg.fit(X_train_st,y_train)
# y_pred=xgb_reg.predict(X_test_st)
# MAE_score=MAE(y_test, y_pred)
# MAE_score
# # print('The MAE score for XgBoost Regression Model is', MAE)

In [97]:
# # Fit a Baseline Model with all features
# # XG Boost
# import xgboost as xgb
# data_train=xgb.DMatrix(data=X_train_st,label=y_train)
# data_test=xgb.DMatrix(data=X_test_st,label=y_test)

# xgb_reg=xgb.XGBRegressor(learning_rate=0.01)

# xgb_reg.fit(X_train_st,y_train)
# y_pred=xgb_reg.predict(X_test_st)
# MAE_score=MAE(y_test, y_pred)
# MAE_score
# # print('The MAE score for XgBoost Regression Model is', MAE)

In [98]:
import xgboost as xgb
data_train=xgb.DMatrix(data=X_train_st,label=y_train)
data_test=xgb.DMatrix(data=X_test_st,label=y_test)

xgb_reg=xgb.XGBRegressor(colsample_bynode=0.8, gamma=0.001,max_depth=6,n_estimators=5)

xgb_reg.fit(X_train_st,y_train)
y_pred=xgb_reg.predict(X_test_st)
MAE_score=MAE(y_test, y_pred)
print(MAE_score)
print(X_train.columns.values)
print(xgb_reg.feature_importances_)

195.42389335326789
['wave_index' 'courier_wave_start_lng' 'courier_wave_start_lat' 'group'
 'level' 'speed' 'grid_distance' 'urgency' 'hour' 'source_type_DELIVERY'
 'source_type_PICKUP' 'weather_grade_Normal Weather'
 'weather_grade_Slightly Bad Weather' 'weather_grade_Very Bad Weather']
[1.1109776e-02 1.3143742e-03 1.1016952e-03 3.3782487e-04 3.6354913e-04
 5.5715553e-03 2.0762073e-01 9.5536508e-02 6.7298920e-03 4.7606575e-01
 1.8716000e-01 0.0000000e+00 0.0000000e+00 7.0883459e-03]


1 The source type still give a good indication

2 It is the 'grid_distance' (it's intuitive because time is postively correlated with distance when speed is set) .  

3 "Very Bad Weather" and "hour of the day" also influence the expected use time as courier may need more time in extreme weather. Also, when they are busy, they tend to spend longer time on each delivery.

4 If the weather is slightly bad, it is not that influential when predicting courier's expected use time.
 

# Prediction

In [99]:
df_p=pd.read_csv('dataframe_test.csv')
df_p.info()
df_p.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25468 entries, 0 to 25467
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   courier_id              25468 non-null  int64  
 1   wave_index              25468 non-null  int64  
 2   tracking_id             25468 non-null  float64
 3   courier_wave_start_lng  25468 non-null  float64
 4   courier_wave_start_lat  25468 non-null  float64
 5   date                    25468 non-null  int64  
 6   group                   25468 non-null  float64
 7   level                   25468 non-null  int64  
 8   speed                   25468 non-null  float64
 9   max_load                25468 non-null  int64  
 10  weather_grade           25468 non-null  object 
 11  aoi_id                  25468 non-null  object 
 12  shop_id                 25468 non-null  object 
 13  id                      25468 non-null  object 
 14  source_type             25464 non-null

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,date,group,level,speed,max_load,...,id,source_type,source_tracking_id,source_lng,source_lat,target_lng,target_lat,grid_distance,urgency,hour
0,10007871,2,2.100000e+18,121.630558,39.1409,20200228,2.020000e+16,3,5.535851,11,...,509790,ASSIGN,2.100000e+18,121.630558,39.140900,121.629930,39.144603,796.0,1776,13
1,10007871,2,2.100000e+18,121.630558,39.1409,20200228,2.020000e+16,3,5.535851,11,...,509791,PICKUP,2.100000e+18,121.629930,39.144603,121.629604,39.150661,1048.0,697,13
2,10007871,2,2.100000e+18,121.630558,39.1409,20200228,2.020000e+16,3,5.535851,11,...,509792,DELIVERY,2.100000e+18,121.629604,39.150661,121.629930,39.144603,1019.0,1756,13
3,10007871,2,2.100000e+18,121.630558,39.1409,20200228,2.020000e+16,3,5.535851,11,...,509793,PICKUP,2.100000e+18,121.629930,39.144603,121.630544,39.152711,1198.0,676,13
4,10007871,2,2.100000e+18,121.630558,39.1409,20200228,2.020000e+16,3,5.535851,11,...,509794,DELIVERY,2.100000e+18,121.630544,39.152711,121.631446,39.142762,1275.0,2336,14


In [100]:
df_p=pd.get_dummies(df_p,columns=['source_type','weather_grade'],drop_first=True)

In [101]:
columns_all=df_p.columns.values
print(columns_all)

columns_need=['wave_index','courier_wave_start_lng','courier_wave_start_lat','group','level','grid_distance','urgency','hour','expected_use_time','speed','weather_grade_Slightly Bad Weather','weather_grade_Very Bad Weather','source_type_DELIVERY','source_type_PICKUP','weather_grade_Normal Weather']
columns_to_drop=[]
for i in columns_all:
    if i not in columns_need:
        columns_to_drop.append(i)

df_p=df_p.drop(columns=columns_to_drop)

['courier_id' 'wave_index' 'tracking_id' 'courier_wave_start_lng'
 'courier_wave_start_lat' 'date' 'group' 'level' 'speed' 'max_load'
 'aoi_id' 'shop_id' 'id' 'source_tracking_id' 'source_lng' 'source_lat'
 'target_lng' 'target_lat' 'grid_distance' 'urgency' 'hour'
 'source_type_DELIVERY' 'source_type_PICKUP'
 'weather_grade_Normal Weather' 'weather_grade_Slightly Bad Weather'
 'weather_grade_Very Bad Weather']


In [102]:
from sklearn.preprocessing import StandardScaler
# We use the mean and standard deviation of the training set to standardize both the training and testing sets.
scaler = StandardScaler().fit(X)
X_st = scaler.transform(X)
df_p=scaler.transform(df_p)

In [103]:
# from sklearn.preprocessing import MinMaxScaler
# scaler=MinMaxScaler()
# scaler.fit(X)
# X_st = scaler.transform(X)
# df_p = scaler.transform(df_p)

In [104]:
Finalized_xgbr=xgb.XGBRFRegressor(colsample_bynode=0.8, gamma=0.001,max_depth=6,n_estimators=5).fit(X_st,y)
predictions=Finalized_xgbr.predict(df_p)
predictions

array([460.34784, 344.99265, 350.9838 , ..., 154.06786, 429.46686,
       615.0924 ], dtype=float32)

In [105]:
predictions=pd.DataFrame(predictions)
predictions.describe()

df_write= pd.read_csv('Regression.csv')
df_write['expected_use_time'] = predictions
df_write

,order,expected_use_time
0,0,460.347839
1,1,344.992645
2,2,350.983795
3,3,360.070068
4,4,455.753204
...,...,...
25463,25463,425.058777
25464,25464,748.078186
25465,25465,154.067856
25466,25466,429.466858


In [106]:
df_write.to_csv(r'Regression_for_sub.csv', index = False, header=True)